In [ ]:
## this version is the same as the one without the 'nodxreq_' prefix except that it
## reads the 'ndr_' (nodxreq) input table, which is simply not restricted to 
## people with a smoking history ** who eventually are diagnosed with lung cancer **
## instead, it's just everyone with a smoking history
## NOTE: output table also has prefix 'ndr'

## EDIT: now further restricted by the WHERE clause in the SQL query, limiting to just reports
## that contain %rads%

In [2]:
# set up a connection to SQL Server and pull some data 
# first, import the required package and define parameters 

import pyodbc  # this is the built-in package for making ODBC connections

# driver
drv = 'ODBC Driver 17 for SQL Server'
# server
svr = 'SRVR ADDR'
# database
db = 'ORD_XXX_XXXX'
# schema
sch = 'Dflt'
#sch = 'Src'

# table 
#tbl = 'sample100k_rad_raw_2018_2022'
#tbl = 'rad_raw_2018_2022'
tbl = 'ndr_rad_raw_2018_2022_all_scans'

#tbl='STIUNotes_TIUDocument_8925'

if 'sample' in tbl:
    print ('this is only a sample of the data!')
    
tblsuff = ''
if 'all_scans' in tbl:
    tblsuff = '_all_scans'

In [7]:
# establish the connection
conn = pyodbc.connect('Driver={%s};Server=%s;Database=%s;Trusted_Connection=yes;' % (drv, svr, db))
cursor = conn.cursor()

cols = "patientICN,examDateTime,radsrc,RadNucMedReportSID,reportText,lungdxdt_minus_repdt,age"

fetchrows = 10
qry = "select count(1) from [%s].[%s].[%s] where reportText like '%%rads%%'" % (db, sch, tbl)

tblrows = int(list(cursor.execute(qry))[0][0])

print ('total records: %s' % str(tblrows))

qry = '''
        select top 20 %s
        from 
        [%s].[%s].[%s] where reporttext is not NULL  
        ''' % (cols, db, sch, tbl)

chunk = [list(r) for r in list(cursor.execute(qry))]

headers = cols.split(',')

print ('\nAll fields except ReportText - first 20 records\n')
for row in chunk:
    print ('\t'.join(map(str,[c for i, c in enumerate(row) if i != headers.index('reportText')])))
    
conn.close()

In [8]:
def parseMeasure(searchpattern, insent, maxdims = 3):
    rawitems = re.findall(searchpattern, insent)
    allmms = []
    for n, raw in enumerate(rawitems):
        raw = re.sub('[^a-zA-Z0-9.]+', '', raw)
        raw = re.sub('by', 'x', raw)
        msplit = raw.split('x')
        if len(msplit) > maxdims:
            msplit = msplit[-maxdims:]
        mmlist = [n + 1]
        mmlist.extend(None for i in range(maxdims))
        for i in range(len(msplit)-1,max(len(msplit)-maxdims-1, -1),-1):
            if 'mm' in msplit[i]: 
                mtype = 'mm'
                msplit[i] = msplit[i][:-2]
            if 'cm' in msplit[i]: 
                mtype = 'cm'
                msplit[i] = msplit[i][:-2]
            mmlist[i+1] = float(msplit[i])
            if mtype == 'cm':
                mmlist[i+1] = mmlist[i+1] * 10
        mmlist.append(insent)
        allmms.append(mmlist)
    return allmms


In [98]:
import re

anyrads = \
re.compile(r'lung\s?-?rads?:?\s?(?:version\s\d\.?\d?\s?|ver\.?\s\d\.?\d?\s?)?(?:category|cat)?\.?:?\s?\s[1-4][a-c|x]?\b')


#getcat = re.compile(r'\b\d[a-c]?\b')
sent = 'reference: lung-rads version 1.1 category 4x incomplete cat 2a assessment categories release date: 2019'
#sent = 'chest CT Lung RADS version 1.0 cat.: 2a and some other stuff'
#sent = 'something else'

lr = re.findall(anyrads, sent.lower())

print (lr)
lr = lr[0]
print (lr[lr.rindex(' '):])
raise
if lr:
    lr = lr[0]
    lr = sent[sent.lower().index(lr) + len(lr):]
    cat = re.findall(getcat, lr)
    print (cat)
    print (lr)

In [93]:
lst = ['a', 'XX', 5]
#ex = [5, 7, None]
#lst.extend([5, 7, None])
#lst.append(5, 7)
#lst.extend([''])
print (lst)

In [1]:
import spacy

In [9]:
import getpass
import re
import nltk

from nltk.tokenize import sent_tokenize, word_tokenize

conn = pyodbc.connect('Driver={%s};Server=%s;Database=%s;Trusted_Connection=yes;' % (drv, svr, db))
cursor = conn.cursor()

cols = "patientICN,examDateTime,radsrc,RadNucMedReportSID,reportText,lungdxdt_minus_repdt,age"

hdict = {}
for i, c in enumerate(cols.split(',')):
    hdict.update({c:i})

sidloc = list(hdict.keys()).index('RadNucMedReportSID')

testonly = True
fetchrows = 100

isfullrun = input('full run? [y/n]: ')

if isfullrun[0].lower() == 'y':
    testonly = False
    fetchrows = 5000

findmass = re.compile(r'tumor|nodule|lesion|\bmass\b|spiculate|opacity|carcinoma|neoplasm')
nonlung = re.compile(r'liver|hepat|kidney|\brenal\b')
replxwalk = [('centimeters?', 'cm'), ('millimeters?', 'mm'), (r'\r\n|\n|\r', ' '), 
            (',', ''), (r'\s+', ' ')]
anymeasure = re.compile('\d+\s?(mm|cm)')

findmmcm = re.compile('((?:\d*\.?\d+\s*(?:mm|cm)*\s*(?:x|by)\s*)*\d*\.?\d+\s*(?:mm|cm))')

#anyrads = re.compile('lung\s?-?rads?:?\s?\n?')
#anyrads = re.compile(r'lung\s?-?rads?:?\s?(?:version\s\d\.?\d?\s?|ver\.?\s\d\.?\d?\s?)?(?:category|cat)?\.?:?\s?')
anyrads = \
re.compile(r'lung\s?-?rads?:?\s?(?:version\s\d\.?\d?\s?|ver\.?\s\d\.?\d?\s?)?(?:category|cat)?\.?:?\s?\s[1-4][a-c|x]?\b')

#getcat = re.compile(r'\d[a-c]?\b')

offset = 0

keeprows = []
#keepReps = []

readrows = tblrows
if testonly:
    readrows = 500

nrowsread = 0
nsentsread = 0
nreports_w_hit = 0
nsents_w_hit = 0
maxsentlen = 0

while offset < readrows:
    qry = '''
        select %s from 
        [%s].[%s].[%s] 
        where reportText like '%%rads%%'
        order by patientICN, RadNucMedReportSID 
        offset %s row fetch next %s rows only
        ''' % (cols, db, sch, tbl, offset, fetchrows)

    chunk = [list(r) for r in list(cursor.execute(qry))]
    nrowsread += len(chunk)
    
    print ('...reading rows %s to %s' % (offset, offset+fetchrows-1))
    
    offset += fetchrows
    
    for row in chunk:
        row[sidloc] = int(row[sidloc])
        anyhit_this_report = 0
        report = row[hdict['reportText']] 
        sections = re.split('\n(?=[A-Z])[A-Z\s\/]+:',report)
        for sec in sections:  
            sentences = sent_tokenize(sec)
            for i, sent in enumerate(sentences):
                has_sdata = False
                nsentsread += 1
                sent = sent.strip().lower()
                for r in replxwalk:
                    sent = re.sub(r[0], r[1], sent)
                lr = re.findall(anyrads, sent.lower())
                if lr:
                    lr = lr[0]
                    lr = lr[lr.rindex(' '):]
                    #lr = sent[sent.lower().index(lr) + len(lr):]
                    #lr = re.findall(getcat, lr)
                    #if lr:
                    #    lr = lr[0]
                if findmass.search(sent) and not nonlung.search(sent): 
                    #sent = sent.strip().lower()
                    #for r in replxwalk:
                    #    sent = re.sub(r[0], r[1], sent)
                    if anymeasure.search(sent): 
                        has_sdata = True
                        sdata = parseMeasure(findmmcm, sent, maxdims = 3)
                if lr or has_sdata:
                    nsents_w_hit += 1
                    maxsentlen = max(maxsentlen, len(sent))
                    rec = row[:]
                    rec.pop(hdict['reportText'])
                    if not anyhit_this_report:
                        #keepReps.append(row[0:5])
                        #keepReps[-1].append(0)
                        anyhit_this_report = 1
                    else:
                        #keepReps[-1][-1] = keepReps[-1][-1] + 1
                        pass
                    if lr:
                        currrec = rec[:]
                        currrec.append(i)
                        #currrec.extend([lr[:10]])
                        currrec.extend([lr])
                        currrec.extend([1, None, None, None, sent]) # measurment data - edit if maxdims is not 3
                        keeprows.append(currrec)
                    if has_sdata:
                        for r in sdata:
                            currrec = rec[:]
                            currrec.append(i) # sentence ID 
                            currrec.extend(['']) # lung RADS
                            currrec.extend(r) # measurement data 
                            keeprows.append(currrec)

        nreports_w_hit += anyhit_this_report
                            
print ('...done')
print ('total reports read:', nrowsread)
print ('total reports w. 1+ hit:', nreports_w_hit)
print ('total sentences read:', nsentsread)
print ('total sentences w. 1+ hit:', nsents_w_hit)
print ('total hits:', len(keeprows))
print ('max sentence length:', maxsentlen)

sentfile = 'ndr_rad_rows_kept%s' % tblsuff
repfile = 'ndr_rad_reports_kept%s' % tblsuff

if testonly:
    sentfile = 'testrun_' + sentfile
    repfile = 'testrun_' + repfile

db = 'ORD_Davies_202110040D'
sch = 'Dflt'
outtbl = sentfile
outcoldict = {
    'patientICN':'VARCHAR(10)',
    'examDateTime':'DATETIME2(0)',
    'radsrc':'VARCHAR(3)',
    'RadNucMedReportSID':'BIGINT',
    'lungdxdt_minus_repdt':'INT',
    'age':'INT',
    'sentID':'INT',
    'lungRADS':'VARCHAR(10)',
    'item':'INT',
    'dim1mm':'DECIMAL',
    'dim2mm':'DECIMAL',
    'dim3mm':'DECIMAL',
    'sent':'VARCHAR(%s)' % maxsentlen
}

outcols = '|'.join(k for k in outcoldict.keys())
outcolsselect = ', '.join(k + ' ' + outcoldict[k] for k in outcoldict.keys())

outcolscsv = outcols.replace('|', ',')

createtbl = 'create table [%s].[%s].[%s] (%s)' % (db, sch, outtbl, outcolsselect)

if not keeprows:
    print ('no rows were selected')
else:
    cursor.execute('drop table if exists [%s].[%s].[%s]' % (db, sch, outtbl))
    cursor.execute(createtbl)

    insertqry = '''
    insert into [%s].[%s].[%s] (%s) VALUES (%s)
    ''' % (db, sch, outtbl, outcolscsv, ','.join('?' for k in outcoldict.keys()))
    
    print (insertqry)
    #print (keeprows[1])
    #raise
    cursor.executemany(insertqry, keeprows)
    conn.commit()
    
    print ('...data were output to the SQL table %s.%s.%s' % (db, sch, outtbl))

cursor.close()
conn.close()

In [28]:
del keeprows

In [30]:
import pandas as pd

#incols = "patientICN,radsrc,examDateTime,radNucMedReportSID,reportText,lungdxdt_minus_repdt"
incols = "patientICN,radsrc,examDateTime,RadNucMedReportSID,lungdxdt_minus_repdt"
incols = '''
patientICN,examDateTime,radsrc,RadNucMedReportSID,lungdxdt_minus_repdt,sentID,item,dim1mm,dim2mm,dim3mm,sent
'''

#incols = cols.split(',')

#outcols = [k for k in outcoldict.keys()]

conn = pyodbc.connect('Driver={%s};Server=%s;Database=%s;Trusted_Connection=yes;' % (drv, svr, db))
cursor = conn.cursor()
tbl = 'rad_rows_kept'

qry = 'select %s from [%s].[%s].[%s]' % (incols, db, sch, tbl)

indata = pd.read_sql(qry, conn)

In [31]:
#print(outcols)
#print(keeprows[0])

#outdata = pd.DataFrame(keeprows, columns = outcols)

In [7]:
'''alldata = pd.merge(
    indata[['patientICN', 'RadNucMedReportSID', 'examDateTime']], 
    outdata, 
    left_on = ['patientICN', 'RadNucMedReportSID'],
    right_on = ['patientICN', 'RadNucMedReportSID'],
    how = 'inner'
)

del indata, outdata'''

In [32]:
indata['repdate'] = pd.to_datetime(indata['examDateTime']).dt.date

indata['max_mm'] = indata[['dim1mm', 'dim2mm', 'dim3mm']].max(axis = 1)

indata.sort_values(by = ['patientICN', 'examDateTime'], ascending = [True, True], inplace = True)

firstrep = indata.groupby('patientICN').nth(0)

print(firstrep.head())







In [33]:
forplot = firstrep[['examDateTime', 'max_mm', 'lungdxdt_minus_repdt']].rename(columns = {'lungdxdt_minus_repdt':'lead_time'})

#forplot.set_index('examDateTime')
#forplot.drop(columns = ['patientICN'])
#forplot.rename(columns={'lungdxdt_minus_repdt':'lead_time'}, inplace = True)
print(forplot.head())

In [61]:
byweek = forplot.resample('1W', on = 'examDateTime').mean()
byweek_count = forplot.resample('1W', on = 'examDateTime').agg({'max_mm':'count'}).rename(columns = {'max_mm':'nppl'})

print (byweek.head())
print (byweek_count.head())

byweek = pd.merge(
    byweek,
    byweek_count,
    left_on = ['examDateTime'],
    right_on = ['examDateTime'],
    how = 'inner'
)

byweek.index.names = ['weekStarting']

del byweek_count
print (byweek.head())

In [62]:
# calculate 12-week weighted moving averages
byweek['sp_mm'] = byweek['max_mm'] * byweek['nppl']
byweek['sp_lt'] = byweek['lead_time'] * byweek['nppl']


In [63]:
rollingavg = byweek[['sp_mm', 'sp_lt', 'nppl']].rolling(12).sum()
rollingavg['sp_mm'] = rollingavg['sp_mm'] / rollingavg['nppl']
rollingavg['sp_lt'] = rollingavg['sp_lt'] / rollingavg['nppl']

byweek = pd.merge(
    byweek[['max_mm', 'lead_time', 'nppl']],
    rollingavg[['sp_mm', 'sp_lt']],
    left_on = ['weekStarting'],
    right_on = ['weekStarting'],
    how = 'left'
)

print (byweek.head(30))